本节我们将超越前面 '效果尚可' 的组合,达到 '效果很好能赢得机器学习比赛' 的程度.

---
前排提醒: 13 章省略了很多文字内容,之后会重写.
---

## 超参数优化

建立一个深度模型时,似乎很多决定都是随意的,
- 堆多少层? 
- 每个层多少单元?
- 激活函数选择?
- ???

这些模型本身的参数称为超参数,区别于模型参数(通过反向传播训练).

超参数本身调节似乎依赖于有经验人员的直觉,没有正式的规则,如果想达到某种任务的极限,最终需要不断的调整,最初的选择是次优的,然后依靠直觉不断调整超参数,直到达到一个临界点---这是机器学习工程师和研究人员花费大部分时间做的事情,但是,作为人类,摆弄超参数这样的工作,最好全留给机器.

---
这就是炼丹

--

我们需要一种原则性的自动调节超参数的方法,自动化的超参数优化是一个很大的研究领域.

通常优化超参数的过程

- 选择一组超参数(自动)
- 建立模型
- 对训练数据拟合,验证数据性能
- 选择下一组超参数尝试(自动)
- 重复,直到性能达到要求.


超参调整过程关键是分析验证性能和各种超参数之间的关系的算法,以选择超参数调整的方向.许多技术都是可行的: 贝叶斯,遗传算法.简单随机搜索.

训练模型的权重是相对容易的: 在小批量上计算损失函数,使用反向传播把权重向正确方向移动.但是更新超参数是另一种完全不同的挑战.

超参数空间是一系列离散的决策组成,因此不是连续且不可微,因此梯度下降无效.必须依赖无梯度的优化技术,但是其效率远远低于梯度下降.

计算优化这个过程的反馈信号(这组超参数是不是对应性能更高)可能非常昂贵,超参数变化意味着在数据集上从头创建和训练模型.

反馈信号本身也并不靠谱,如果一组超参数的效果好了 0.2%,那是超参数更好了,还是初始权重的参数更适合了?

虽然很麻烦,幸运的是确实有工具帮助超参数调整--KerasTuner

先安装

In [1]:
!pip install keras-tuner -q

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


KerasTuner 的想法是用一组可能的超参数调整空间,取代超参数的硬编码,例如不是 `units=32` 而是 `Int(name="units", min_value=16, max_value=64, step=16)`.

为了指定一个超参数的搜索空间,需要定义一个返回模型的函数,有一个 hp 函数,代表超参数的搜索空间.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16) #从hp获取超参数值
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"]) #有不同种类的超参数数值
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

如果需要更加模块化和更高的可配置性,可以继承 HyperModel 实现子类,定义一个构建方法.

In [3]:
import kerastuner as kt


class SimpleMLP(kt.HyperModel):
    def __init__(self,
                 num_classes):  #面向对象的方法，我们可以将模型常量配置为构造函数参数（而不是在模型建立函数中硬编码
        self.num_classes = num_classes

    def build(self, hp):  #构建方法与我们之前的build_model独立函数相同
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax"
                         )  #面向对象的方法，我们可以将模型常量配置为构造函数参数（而不是在模型建立函数中硬编码
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(optimizer=optimizer,
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])
        return model


hypermodel = SimpleMLP(num_classes=10)

下一步是需要定义一个调谐器,原理上是个 for 循环

- 选择一组超参
- 建立对应模型
- 训练模型记录指标

keras 有几个内置的调谐器 RandomSearch、BayesianOptimization、Hyperband,先来试试 `BayesianOptimization`



In [4]:
tuner = kt.BayesianOptimization(
    build_model,#模型建立函数
    objective="val_accuracy",#调谐器寻求优化的指标
    max_trials=100,#结束前,尝试的最大次数
    executions_per_trial=2,#减少参数随机性,同一超参训练2次,指标取平均
    directory="mnist_kt_test",#log 存储
    overwrite=True,#修改 build_model 后需要设置为 true,否则会接着上次的搜索
)

2021-11-10 11:54:27.436677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 11:54:27.442660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 11:54:27.442994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 11:54:27.444265: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

search_space_summary 可以显示搜索空间的概况

In [5]:
tuner.search_space_summary()


Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


----

内置的指标(例如上面的准确度),到底是最大化还是最小化,KerasTuner能自动推断,但是自定义的度量需要自己指定.


```py
objective = kt.Objective(
    name="val_accuracy",#指标名称
    direction="max")# min or max
tuner = kt.BayesianOptimization(
    build_model,
    objective=objective,
    ...
)
```

----

开启我们的搜索

- 传递验证数据(前万别是测试集)

In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]  #留待后用
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]  #预留验证集
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5),  #对应大量的 epochs,使用 EarlyStopping 回调,过拟合时停止训练
]
tuner.search(  #传递与 fit 相同的参数
    x_train,
    y_train,
    batch_size=128,
    epochs=100,  #大量的 epochs
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 29 Complete [00h 01m 20s]
val_accuracy: 0.9752500057220459

Best val_accuracy So Far: 0.9760500192642212
Total elapsed time: 00h 43m 39s

Search: Running Trial #30

Hyperparameter    |Value             |Best Value So Far 
units             |64                |64                
optimizer         |rmsprop           |rmsprop           

Epoch 1/100
391/391 - 2s - loss: 0.4165 - accuracy: 0.8883 - val_loss: 0.2234 - val_accuracy: 0.9387
Epoch 2/100
391/391 - 2s - loss: 0.2104 - accuracy: 0.9397 - val_loss: 0.1765 - val_accuracy: 0.9513
Epoch 3/100
391/391 - 2s - loss: 0.1627 - accuracy: 0.9531 - val_loss: 0.1513 - val_accuracy: 0.9581
Epoch 4/100
391/391 - 2s - loss: 0.1332 - accuracy: 0.9612 - val_loss: 0.1399 - val_accuracy: 0.9620
Epoch 5/100
391/391 - 2s - loss: 0.1141 - accuracy: 0.9677 - val_loss: 0.1206 - val_accuracy: 0.9668
Epoch 6/100
391/391 - 2s - loss: 0.0991 - accuracy: 0.9718 - val_loss: 0.1187 - val_accuracy: 0.9664
Epoch 7/100
391/391 - 2s - loss: 0.0878 - accuracy:

上面的例子只需要几分钟完成,因为可能的选择只有几种,且数据集是 helloworld 的 minist,但是实践中这个过程可能长达一整夜甚至几天.

当搜索崩溃时,调用时别忘了 `overwrite=False` 这样就能从日志中恢复训练过程

一旦搜索完成,最佳的超参对应高性能的模型.

查询最佳超参

In [ ]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n) #会返回超参list,可以传递给模型建立函数

超参选择时保留的验证数据,已经不会调整超参了,在重新训练模型一般会重新加入训练数据.

当在构建最终模型时,还有一个参数,训练的轮次,是可以设置一个很大的轮次,然后使用 EarlyStopping 回调,但是这样浪费时间还有可能导致模型没有处于最佳状态.

幸运的是这个轮次我们只需要使用验证集寻找就行了

In [ ]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks = [
        keras.callbacks.EarlyStopping(monitor="val_loss",
                                      mode="min",
                                      patience=10)  #注意 patience 非常高
    ]
    history = model.fit(x_train,
                        y_train,
                        validation_data=(x_val, y_val),
                        epochs=100,
                        batch_size=128,
                        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

最后在完整的数据集上训练,因为加入了验证数据所以训练时间要额外长出 20%

In [ ]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(x_train_full,
              y_train_full,
              batch_size=128,
              epochs=int(best_epoch * 1.2))
    return model


best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

如果担心这样训练可能导致模型稍微差一点(参数的随机性),可以走个捷径,直接使用调制过程中表现最好的模型,而不需要重新训练

In [ ]:
best_models = tuner.get_best_models(top_n)

---
注意:

在进行大规模的自动超参数优化时，需要注意的一个重要问题是验证集过拟合。因为你是根据使用验证数据计算的信号来更新超参数，你实际上是在验证数据上训练它们，因此它们会很快对验证数据过度拟合。永远记住这一点。

---

### 构建正确搜索空间

---

简略

---


总体而言,超参数优化是个强大的技术.但是并不能代替你对模型本身的熟悉,仍然需要手动挑选可能生成最好超参的配置.

好消息是,KerasTuner 使得我们可以不用纠结在各种微观参数上,把握宏观(是不是要用残差链接?...)而且每一类问题的调整都比较通用.

图像分类 -> kt.applications.HyperXception 或者 kt.applications.HyperResNet

### 超参调整未来: 自动化机器学习

---

无翻译,纯吐槽

AutoML 自动化机器学习,当这一天降临时请千万不要惊讶.

着眼大局,而不是变成调参侠.

---


## 组合模型

单一模型是有极限的,因此开始组合 1+1 >? 2 了.但是等式真的成立吗?


下面是对分类器的平均,但是恐怕只在各个分类器性能都差不多时才能成立.

```py
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)
```

更好的做法是进行加权平均,性能好的权重高.
```py
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d
```

有许多组合模型的方式,简单预测值的平均,加权平均等.

组合模型需要保存分类器足够的多样性,每个分类器的偏见各不相同,最终的组合就可能准确的.

有一个是事实但不是万能的: 基于树的方法(随机森林)和深度神经网络的组合效果很好.

## 扩大模型训练规模

随着对 keras api 的熟悉,模型编码速度不再是瓶颈,下一个瓶颈是模型的训练速度.更快的训练意味着更快的调整.(有点像敏捷开发)

本节中3种方法加快训练

- 混合精度训练,甚至只在一个 gpu 上训练
- 多GPU 组合训练
- TPU 训练

### 混合精度训练

缩减浮点数精度,可以成倍缩减训练时间
- float16 半精度
- float32 单精度
- float64 双精度

面对精度下降导致效果下降 -> 混合使用 float16/float32/float64

效果: 基本不影响效果情况下,GPU 训练速度x3 TPU 速度 +60%

小心 dtype 的默认值

Keras 和 tensorflow 默认是单精度,但是 NumPy 默认是双精度 float64,白白浪费空间

记得一定要转换:

```py
>>> np_array = np.zeros((2, 2))
>>> tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
>>> tf_tensor.dtype
tf.float32
```

### 混合精度实践

打开混合精度

In [1]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2060, compute capability 7.5


2021-11-10 12:56:03.261507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 12:56:03.269126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 12:56:03.269457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 12:56:03.270260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


通常模型的前向传递是 float16,模型权重在 float32.

Keras层 有一个variable_dtype和一个compute_dtype属性

- 默认两者都是 float32,
- 混合精度,大部分compute_dtype 变成 float16,但 variable_dtype 仍然是 float32 因此权重依然能接收 flaot32 更新.

注意一些操作在 float16 上是不稳定的,需要明确向层传入不使用混合精度 `dtype="float32"` 传递给层的构造函数.

### 多GPU训练

多 GPU 训练有两种方式

- 模型并行
- 数据并行

模型并行暂时不涉及,只看数据并行

#### 获取更多 GPU

最好租用虚拟GPU

#### 单主机多设备同步训练

一旦有了多 GPU,导入 tensorflow 就可以开始分布式训练了

In [3]:
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()  # 创建分配策略对象 MirroredStrategy应该是首选
print(f"Number of devices: {strategy.num_replicas_in_sync}")
with strategy.scope():  # 打开一个策略范围
    model = get_compiled_model()  # 创建变量等都应该在这个范围内
model.fit(train_dataset,
          epochs=100,
          validation_data=val_dataset,
          callbacks=callbacks)  #在所有可用设备上训练


2021-11-10 13:03:18.451075: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-10 13:03:18.453593: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 13:03:18.453870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-10 13:03:18.454195: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built witho

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


NameError: name 'get_compiled_model' is not defined

上面那几行就实现了: 单设备 多主机的同步训练,也被称为镜像训练策略.

当你打开一个MirroredStrategy范围并在其中建立你的模型时，MirroredStrategy对象将在每个可用的GPU上创建一个模型副本（replica）。然后，训练的每一步都以如下方式展开（见图13.3）。

从数据集中抽取一批数据（称为全局批）。

它被分割成4个不同的子批（称为本地批）。例如，如果全局批有512个样本，4个局部批中的每一个将有128个样本。因为你希望本地批处理足够大，以保持GPU的忙碌，所以全局批处理的大小通常需要非常大。


4个副本中的每一个都在自己的设备上独立地处理一个本地批处理：它们运行一个前向传递，然后是后向传递。每个副本输出一个 "权重delta"，描述在模型中更新每个权重变量的程度，即以前的权重梯度与本地批处理的模型损失的关系。


源自本地梯度的权重delta被有效地合并到4个副本中，以获得一个全局delta，它被应用于所有副本。因为这是在每一步结束时进行的，所以各副本总是保持同步：它们的权重总是相等的。

tf.data性能提示

在进行分布式训练时，始终以tf.data.Dataset对象的形式提供数据，以保证最佳性能（以NumPy数组形式传递数据也行，因为这些数据会被fit()转换为Dataset对象）。你还应该确保利用数据预取：在将数据集传递给fit()之前，调用dataset.prefetch(buffer_size)。如果你不确定该选择什么缓冲区大小，可以试试dataset.prefetch(tf.data.AUTOTUNE)选项，它将为你选择一个缓冲区大小。


在一个理想的世界里，在N个GPU上训练会带来N倍的速度提升。然而，在实践中，分布引入了一些开销--特别是合并来自不同设备的权重德尔塔需要一些时间。你得到的有效速度是使用的GPU数量的一个函数。

- 使用两个GPU，速度会保持在2倍左右。
- 使用四个，速度提高了3.8倍左右。
- 如果是8个，则是7.3倍左右。

这假定你使用了足够大的全局批处理量，以保持每个GPU的满负荷使用。如果你的批处理量太小，本地批处理量将不足以让你的GPU保持忙碌。


### TPU训练

Colab 提供了 TPU 选择,使用时需要多一个连接到 TPU 集群选项

无需多想,直接执行就行

```py
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print("Device:", tpu.master())
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
```

与多 GPU 训练相似,需要打开分布式策略,这里是 `TPUStrategy` 步骤完全相同

```py
from tensorflow import keras
from tensorflow.keras import layers

strategy = tf.distribute.TPUStrategy(tpu)
print(f"Number of replicas: {strategy.num_replicas_in_sync}")

def build_model(input_size):
    inputs = keras.Input((input_size, input_size, 3))
    x = keras.applications.resnet.preprocess_input(inputs)
    x = keras.applications.resnet.ResNet50(
        weights=None, include_top=False, pooling="max")(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

with strategy.scope():
    model = build_model(input_size=32)
```

colab TPU 有点奇怪,是双虚拟机,这意味着不能直接读本地硬盘,两种办法

- 直接放在内存
- 放在google云存储, google drive 等

TPU 完成第一次历时之后,就会快的惊人.此时读取数据反而称为瓶颈了.

- 全放内存,调用 `dataset.cache()`
- 内存放不下,把格式转换成 `TFRecord`

#### 提供 tpu 利用率

说白了要保存 tpu 都忙起来,需要设置巨量的批次大小 超过 10000,相应的提供学习率,确保能跟上.

另一个技巧: 步骤融合,一次执行多个训练步骤.

compile()中指定 step_per_execution,例如 step_per_execution=8 ,每个 tpu 周期执行 8 步,充分利用 tpu.